In [1]:
import sys
sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'/home/ikwak2/hmd/notebooks/utils')


In [2]:
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import *
import pickle as pk



In [3]:
root_dir = '/home/ikwak2/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'

In [4]:
data_folder = training_data_dir

In [5]:
model_folder = '/home/ikwak2/hmd/tmpmodel'

In [6]:
verbose = 1

In [7]:
#data_folder, model_folder, verbose

In [8]:
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

In [9]:
    os.makedirs(model_folder, exist_ok=True)

    classes = ['Present', 'Unknown', 'Absent']
    num_classes = len(classes)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    features1 = get_features(data_folder, patient_files)
    model = get_toy((100, 313, 1))
    model.compile(optimizer = "adam", 
             loss = "categorical_crossentropy",
             metrics = "accuracy")
    

Extracting features and labels from the Challenge data...


In [10]:
import pandas as pd

In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer().fit(features1[0]['hw'])
features1[0]['hw'] = imputer.transform(features1[0]['hw'])

In [12]:
features1

({'id': array(['2530', '2530', '2530', ..., '85349', '85349', '85349'],
        dtype='<U5'),
  'age': array([[0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         ...,
         [0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 1]]),
  'sex': array([[1, 0],
         [1, 0],
         [1, 0],
         ...,
         [1, 0],
         [1, 0],
         [1, 0]]),
  'hw': array([[ 98.        ,  15.9       ],
         [ 98.        ,  15.9       ],
         [ 98.        ,  15.9       ],
         ...,
         [115.00534188,  24.84538426],
         [115.00534188,  24.84538426],
         [115.00534188,  24.84538426]]),
  'preg': array([False, False, False, ...,  True,  True,  True]),
  'loc': array([[1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         ...,
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.]]),
  'mel1': array([[[[-13.808031  ],
           [-14.46

In [13]:
model.fit([features1[0]['age'],features1[0]['sex'], features1[0]['hw'], features1[0]['preg'], features1[0]['loc'], 
           features1[0]['mel1']], features1[1],
          epochs = 2)

Epoch 1/2
99/99 [==============================] - 22s 221ms/step - loss: 1.1953 - accuracy: 0.7218
Epoch 2/2
99/99 [==============================] - 22s 221ms/step - loss: 0.6779 - accuracy: 0.7562


In [14]:
def save_challenge_model(model_folder, model, classes, m_name, mel_shape = (100, 313, 1)) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, m_name + '_model.hdf5')
    model.save(filename)
    d = {'model': m_name, 'classes': classes, 'mel_shape': mel_shape, 'model_fnm': filename}    
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)

In [15]:
    save_challenge_model(model_folder, model, classes, m_name = 'toy', mel_shape = (100, 313, 1) )
    if verbose >= 1:
        print('Done.')


Done.


In [16]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [17]:
model1 = load_challenge_model(model_folder, verbose = 1) ### Teams: Implement this function!!!

In [18]:
model1

{'model': 'toy',
 'classes': ['Present', 'Unknown', 'Absent'],
 'mel_shape': (100, 313, 1),
 'model_fnm': '/home/ikwak2/hmd/tmpmodel/toy_model.hdf5'}

In [19]:
data_folder

'/home/ikwak2/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1/training_data'

In [20]:
model1

{'model': 'toy',
 'classes': ['Present', 'Unknown', 'Absent'],
 'mel_shape': (100, 313, 1),
 'model_fnm': '/home/ikwak2/hmd/tmpmodel/toy_model.hdf5'}

In [24]:
recordings = load_recordings(data_folder, patient_data)

NameError: name 'patient_data' is not defined

In [73]:
recordings.shape

AttributeError: 'list' object has no attribute 'shape'

In [72]:
len(recordings[0])

94400

In [83]:
string1 = 'dfsdfsda'

In [86]:
isinstance(string1, str)

True

In [36]:
def run_challenge_model(model, data, recordings, verbose):
    
    if model['model'] == 'toy' :
        model1 = get_toy(model['mel_shape'])
    filename = os.path.join(model_folder, model['model'] + '_model.hdf5')
    model1.load_weights(filename)
    
    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose)

    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i])[0]
        features['mel1'].append(mel1)

    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
        
    features['mel1'] = np.array(features['mel1'])
#    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
    prob1 = res1.mean(axis = 0) ## simple rule for now
    idx = np.argmax(prob1)
    # Choose label with higher probability.
    labels = np.zeros(len(classes), dtype=np.int_)
    labels[idx] = 1

    return classes, labels, prob1

In [37]:
def get_feature_one(patient_data, verbose = 0) :

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_locations = get_num_locations(patient_data)
    recording_information = patient_data.split('\n')[1:num_locations+1]

    features = dict()
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
#    features['mel1'] = []
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
#        filename = os.path.join(data_folder, recording_file)

        # Extract id
    #    id1 = recording_file.split('_')[0]
    #    features['id'].append(id1)

        # Extract melspec
#        mel1 = feature_extract_melspec(filename)[0]
#        features['mel1'].append(mel1)

        # Extract age_group
        age_group = get_age(patient_data)
        current_age_group = np.zeros(6, dtype=int)
        if age_group in age_classes:
            j = age_classes.index(age_group)
            current_age_group[j] = 1
        else :
            current_age_group[5] = 1
        features['age'].append(current_age_group)

        # Extract sex
        sex = get_sex(patient_data)
        sex_features = np.zeros(2, dtype=int)
        if compare_strings(sex, 'Female'):
            sex_features[0] = 1
        elif compare_strings(sex, 'Male'):
            sex_features[1] = 1
        features['sex'].append(sex_features)

        # Extract height and weight.
        height = get_height(patient_data)
        weight = get_weight(patient_data)
        features['hw'].append(np.array([height, weight]))

        # Extract pregnancy
        is_pregnant = get_pregnancy_status(patient_data)
        features['preg'].append(is_pregnant)

        # Extract location
        locations = entries[0]
        num_recording_locations = len(recording_locations)
        loc_features = np.zeros(num_recording_locations)
        if locations in recording_locations:
            j = recording_locations.index(locations)
            loc_features[j] = 1
        features['loc'].append(loc_features)
        

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
        
    if verbose :
        label = get_label(patient_data)
        print(label)
    return features

In [38]:
patient_data = load_patient_data(patient_files[0])
recordings = load_recordings(data_folder, patient_data)
run_challenge_model(model1, data = patient_data, recordings = recordings, verbose = 1)

Absent


(['Present', 'Unknown', 'Absent'],
 array([1, 0, 0]),
 array([9.999956e-01, 4.065800e-10, 4.428094e-06], dtype=float32))

In [39]:
patient_data = load_patient_data(patient_files[1])
recordings = load_recordings(data_folder, patient_data)
run_challenge_model(model1, data = patient_data, recordings = recordings, verbose = 1)

Present


(['Present', 'Unknown', 'Absent'],
 array([1, 0, 0]),
 array([9.9999857e-01, 6.0714815e-11, 1.4679399e-06], dtype=float32))